<span>
<img src="http://ndlib.readthedocs.io/en/latest/_static/ndlogo2.png" width="240px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti</a><br/>
<b>Python version:</b>  3.7<br/>
<b>NDlib version:</b>  4.0.1<br/>
<b>Last update:</b> 30/07/2020
</span>

<a id='top'></a>
# *Intro to NDlib-REST: Remote experiment definition and execution*

[``NDlib``](http://ndlib.readthedocs.io) is a python library designed to provide support to analysis of diffusive phenomena occurring on top of complex network structures.

[``NDlib-REST``](http://ndlib.readthedocs.io/en/latest/rest/ndlib-rest.html) is a restful server that allows to decouple experiment definition (client side) and execution (server side). 

In this notebook is introduced the python endpoint workflow to setup a diffusion experiment using the [``NDlib-REST``](http://ndlib.readthedocs.io/en/latest/rest/ndlib-rest.html) API.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started.

## Table of Contents

1. [Setup the Experiment Server](#install)
2. [Experiment definition and execution](#workflow)

<a id='install'></a>
## 1. Setup the Experiment Server ([to top](#top))

**1)** Move to the working directory

In [2]:
!rm -rf ndlib-rest

**2)** Clone the ``NDlib-REST`` server project from the project [GitHub](https://github.com/GiulioRossetti/ndlib-rest) 

In [3]:
!git clone https://github.com/GiulioRossetti/ndlib-rest.git > /dev/null

Cloning into 'ndlib-rest'...
remote: Enumerating objects: 1325, done.
remote: Total 1325 (delta 0), reused 0 (delta 0), pack-reused 1325
Receiving objects: 100% (1325/1325), 8.20 MiB | 496.00 KiB/s, done.
Resolving deltas: 100% (757/757), done.


**3)** Start the REST server

In [4]:
%cd ndlib-rest/
import subprocess as sub
sub.call('./gunicorn.sh', shell=True)

/Users/pietromonticone/github/CNA_Tutorials/ndlib-rest


127

<a id='workflow'></a>
## 2. Client for remote experiment setup ([to top](#top))

``NDlibClient``, installed along with the REST server, provides a python wrapper around the endpont calls offered by ``NDlib-REST``. <br/>

For endpoint parameters specs refer to the <a href="/docs">Service API</a>.

In [5]:
from client.NDlibClient import NDlibClient

**1)** Specify the base url of the server exposing the rest API

In [10]:
exp = NDlibClient("http://127.0.0.1:5000")

**2)** Create an experiment

In [11]:
exp.create_experiment()

KeyError: 'token'

**3)** Add a graph resource to the experiment

In [12]:
resp = exp.add_erdos_renyi_graph(1000, 0.1)

**4)** Add one or more diffusion models to the experiment

In [13]:
resp = exp.add_SIR(infected=0.01, beta=0.001, gamma=0.09)

**5)** Execute the simulation

In [14]:
it = exp.get_iteration_bunch(bunch=15)

**6)** Analyze the results

In [15]:
models = it.keys()

In [16]:
for m in models:
    print("Model: %s\n" % m)
    for i in range(0, len(it[m])):
        print("Iteration: %s\t Infected: %s\t Susceptible: %s\t Removed: %s" % (it[m][i]['iteration'], 
         it[m][i]['node_count']['1'], 
         it[m][i]['node_count']['0'], 
         it[m][i]['node_count']['2']))

Model: Message



TypeError: string indices must be integers

**7)** Destroy the experiment and free the assigned resources

In [17]:
resp = exp.destroy_experiment()